# Acceso a Bases de Datos

Para conectarnos desde nuestro código Java a una base de datos necesitamos un driver que nos permita realizar la conexión. En este caso vamos a utilizar el driver de MySQL.

El driver de MySQL se puede descargar desde la página oficial de MySQL: [https://dev.mysql.com/downloads/connector/j/](https://dev.mysql.com/downloads/connector/j/). Nos descargaremos la versión "Platform Independent" en formato ZIP.

Una vez descargado el driver, lo descomprimimos y escogemos el fichero `mysql-connector-java-8.0.23.jar` haciendo botón derecho sobre las librerías de nuestro proyecto y seleccionando "Add JAR/Folder".

El proceso para conectarnos a una base de datos MySQL desde Java es el siguiente:

- Establecer los datos de conexión y conectarnos a la base de datos mediante un objeto `Connection`.
- Ejecutar las consultas SQL mediante un objeto `PreparedStatement`.
- Tratar los resultados obtenidos mediante un objeto `ResultSet`.
- Cerrar la conexión con la base de datos y otros objetos utilizados.

## Conexión

Para conectarnos a una base de datos MySQL necesitamos los siguientes datos:

- `url`: URL de la base de datos.
- `user`: Usuario de la base de datos.
- `password`: Contraseña del usuario de la base de datos.

In [ ]:
import java.sql.*;

Connection conexion = DriverManager.getConnection("jdbc:mysql://localhost:3306/pruebas", "root", "password");

Debemos capturar las posibles excepciones que puedan producirse al intentar establecer la conexión.

In [ ]:
import java.sql.*;

try {
    Connection conexion = DriverManager.getConnection("jdbc:mysql://localhost:3306/pruebas", "root", "password");
    conexion.close(); //Al finalizar, cerramos la conexión
} catch (SQLException e) {
    System.out.println("Error al conectar a la base de datos: " + e.getMessage());
}

Tras realizar nuestras consultas a la base de datos, debemos cerrar la conexión, para ello utilizamos el método `close()`.

Aunque podríamos evitarlo si utilizamos la construcción de Java `try-with-resources` que cierra automáticamente los recursos abiertos al finalizar el bloque `try`. En esta construcción, todos los recursos que se abran dentro del paréntesis de apertura del `try` se cerrarán automáticamente al finalizar el bloque `try`.

In [ ]:
try (Connection conexion = DriverManager.getConnection("jdbc:mysql://localhost:3306/pruebas", "root", "password")) {
    //Aquí dentro realizamos las operaciones con la base de datos
} catch (SQLException e) {
    System.out.println("Error al conectar a la base de datos: " + e.getMessage());
}

# Operaciones CRUD

Las operaciones CRUD son las operaciones básicas que se pueden realizar sobre una base de datos: Create, Read, Update y Delete.

Para ello, utilizaremos la clase `PreparedStatement`. También existe la clase `Statement`, pero es más seguro utilizar `PreparedStatement` ya que evita la inyección SQL, por lo que solo utilizaremos esta última.

Un objeto de esta clase se genera de la siguiente manera:

```java
PreparedStatement ps = conexion.prepareStatement(sentenciaSQL);
```

Esta clase dispone de dos métodos para ejecutar consultas SQL: 

- `executeQuery()`. Se utiliza para consultas de tipo `SELECT` y devuelve un objeto `ResultSet` con los resultados obtenidos.
- `executeUpdate()`. Se utiliza para consultas de tipo `INSERT`, `UPDATE` y `DELETE` y devuelve un entero con el número de filas afectadas.

In [ ]:
String sql = "SELECT * FROM Alumnos";

try (Connection conexion = DriverManager.getConnection("jdbc:mysql://localhost:3306/pruebas", "root", "password");
     PreparedStatement sentencia = conexion.prepareStatement(sql);
     ResultSet resultado = sentencia.executeQuery()) {
    while (resultado.next()) {
        System.out.println(resultado.getString("nombre"));
    }
} catch (SQLException e) {
    System.out.println("Error al conectar a la base de datos: " + e.getMessage());
}

In [ ]:
String sql = "DELETE FROM city WHERE CountryCode='ESP'";

        try (Connection conexion = DriverManager.getConnection("jdbc:mysql://localhost:3306/world", "root", "renaido");
             PreparedStatement sentencia = conexion.prepareStatement(sql);) {
            int numFilas = sentencia.executeUpdate();
            System.out.println(numFilas + " filas borradas");
        } catch (SQLException e) {
            System.out.println("Error al conectar a la base de datos: " + e.getMessage());
        }

## Manejo del `ResultSet`

Un objeto `ResultSet` es una estructura en la que se almacenan los datos que devuelve la consulta. Se puede acceder a dichos datos mediante los siguientes métodos:

- `next()`. Se utiliza para avanzar al siguiente registro. Devuelve `true` si hay más registros y `false` si no hay más registros.
- `getTipoDato("nombreColumna")`. Se utiliza para obtener el valor de una columna. El tipo de dato debe ser el adecuado para la columna que se está consultando.
- `getTipoDato(posiciónColumna)`. Se utiliza para obtener el valor de una columna. El tipo de dato debe ser el adecuado para la columna que se está consultando. En este caso, la primera columna es la 1.
- `getRow()`. Devuelve el número de fila actual.

## Cierre de objetos

Es importante cerrar los objetos `PreparedStatement`, `ResultSet` y `Connection` una vez que hayamos terminado de utilizarlos. Para ello, utilizamos el método `close()`.

En el caso de que los hayamos creado dentro de una cláusula `try-with-resources`, no será necesario cerrarlos ya que se cerrarán automáticamente al finalizar el bloque `try`.

## Consultas preparadas (parametrización de consultas)

Las consultas preparadas son consultas SQL que contienen parámetros. Estos parámetros se representan con el símbolo `?` y se sustituyen por los valores correspondientes mediante el método `setTipoDato(posición, valor)`. el primero de los parámetros tendrá la posición 1.

Estas son las principales ventajas de utilizar consultas preparadas:

- Evitan la inyección SQL.
- Mejoran el rendimiento de la consulta.
- Permiten reutilizar la consulta con diferentes valores.

In [ ]:
String sql = "SELECT * FROM Alumnos WHERE YEAR(fecha_nacimiento) = ?;";

try (Connection conexion = DriverManager.getConnection("jdbc:mysql://localhost:3306/pruebas", "root", "renaido"); 
        PreparedStatement sentencia = conexion.prepareStatement(sql);) {
    sentencia.setInt(1, 2005);
    try (ResultSet resultado = sentencia.executeQuery()) {
        while (resultado.next()) {
            System.out.println(resultado.getString("nombre"));
        }
    }
} catch (SQLException e) {
    System.out.println("Error al conectar a la base de datos: " + e.getMessage());
}

In [ ]:
// Repetición de consultas

String sql = "SELECT * FROM Alumnos WHERE YEAR(fecha_nacimiento) = ?;";

try (Connection conexion = DriverManager.getConnection("jdbc:mysql://localhost:3306/pruebas", "root", "renaido"); 
        PreparedStatement sentencia = conexion.prepareStatement(sql);) {
    for (int anho = 2000; anho <= 2010; anho++) {
    sentencia.setInt(1, anho);
        System.out.println("Año: " + anho);
        try (ResultSet resultado = sentencia.executeQuery()) {
            while (resultado.next()) {
                System.out.println(resultado.getString("nombre"));
            }
        }
    }
} catch (SQLException e) {
    System.out.println("Error al conectar a la base de datos: " + e.getMessage());
}

In [ ]:
String sql = "SELECT * FROM Alumnos WHERE YEAR(fecha_nacimiento) = ? and apellido LIKE ?";

try (Connection conexion = DriverManager.getConnection("jdbc:mysql://localhost:3306/pruebas", "root", "renaido"); PreparedStatement sentencia = conexion.prepareStatement(sql);) {

    sentencia.setInt(1, 2005);
    sentencia.setString(2, "R%");

    try (ResultSet resultado = sentencia.executeQuery()) {
        while (resultado.next()) {
            System.out.println(resultado.getString("nombre"));
        }
    }

} catch (SQLException e) {
    System.out.println("Error al conectar a la base de datos: " + e.getMessage());
}

- Aunque los parámetros sean textos o fechas, no tenemos que preocuparnos de las
comillas en la consulta, al sustituir la interrogación por el valor, JDBC se encarga
de ello.
- Si el parámetro es una fecha, el valor debe ser de tipo `java.sql.Date`, no `LocalDate`.

In [ ]:
LocalDate fecha = LocalDate.now();
ps.setDate(1, java.sql.Date.valueOf(fecha));

## Inserción, modificación y eliminación de registros

Para insertar, modificar o eliminar registros de una base de datos, utilizamos el método `executeUpdate()`. Este método devuelve un entero con el número de filas afectadas.

In [ ]:
String sql = "INSERT INTO Alumnos (nombre, apellido, fecha_nacimiento) VALUES (?, ?, ?)";

try (Connection conexion = DriverManager.getConnection("jdbc:mysql://localhost:3306/pruebas", "root", "renaido"); PreparedStatement sentencia = conexion.prepareStatement(sql);) {

    sentencia.setString(1, "Fernando");
    sentencia.setString(2, "Gómez");
    LocalDate fecha = LocalDate.of(2004, 4, 13);
    sentencia.setDate(3, Date.valueOf(fecha));

    int cantidadFilas = sentencia.executeUpdate();
    System.out.printf("Insertadas %d filas\n", cantidadFilas);

} catch (SQLException e) {
    System.out.println("Error al conectar a la base de datos: " + e.getMessage());
}

In [ ]:
// Repetición de consultas
sql = "SELECT Name FROM country WHERE Population >= ? and Population < ?";

try (Connection conexion = DriverManager.getConnection("jdbc:mysql://localhost:3306/world", "root", "renaido"); 
        PreparedStatement sentencia = conexion.prepareStatement(sql);) {
    for (int limite = 1; limite <= 100000; limite *= 10) {
        sentencia.setInt(1, limite);
        int limiteSuperior = limite * 10;
        sentencia.setInt(2, limiteSuperior);
        System.out.println("");
        System.out.println("Entre " + limite + " y " + limiteSuperior);
        try (ResultSet resultado = sentencia.executeQuery()) {
            while (resultado.next()) {
                System.out.println(resultado.getString("Name"));
            }
        }
    }
} catch (SQLException e) {
    System.out.println("Error al conectar a la base de datos: " + e.getMessage());
}

## Transacciones

Una transacción es un conjunto de operaciones que se ejecutan como una sola unidad. Si una de las operaciones falla, se deshacen todas las operaciones realizadas hasta ese momento. Este tipo de operaciones se conocen como **operaciones atómicas** y se utilizan para garantizar la integridad de los datos.

Para trabajar con transacciones en Java, utilizamos los métodos:
- `setAutoCommit(false)`. Deshabilita el modo de autocommit. Habitualmente, las operaciones se realizan en modo autocommit, es decir, cada operación se confirma automáticamente.
- `commit()`. Confirma la transacción.
- `rollback()`. Deshace la transacción. 

In [ ]:
String url = "jdbc:mysql://localhost:3306/nombre_base_datos";
String user = "usuario";
String password = "contraseña";

try (Connection conn = DriverManager.getConnection(url, user, password)) {
    conn.setAutoCommit(false); // Deshabilitar auto-commit

    try (PreparedStatement sentencia = conn.prepareStatement("INSERT INTO Alumnos (nombre, apellido, fecha_nacimiento) VALUES (?, ?, ?)")) {

        sentencia.setString(1, "Juan");
        sentencia.setString(2, "Pérez");
        sentencia.setDate(3, java.sql.Date.valueOf("2005-09-01"));
        sentencia.executeUpdate();

        sentencia.setString(1, "María");
        sentencia.setString(2, "González");
        sentencia.setDate(3, java.sql.Date.valueOf("2006-10-15"));
        sentencia.executeUpdate();

        conn.commit(); // Confirmar la transacción
    } catch (SQLException e) {
        conn.rollback(); // Deshacer la transacción en caso de error de la BD
        throw e;
    } catch (Exception e) {
        conn.rollback(); // Deshacer la transacción en caso de otro error
} catch (SQLException e) {
    e.printStackTrace();
}
    

## Patrón DAO

El patrón DAO (Data Access Object) es un patrón de diseño que se utiliza para abstraer y encapsular el acceso a la base de datos. De esta forma, separamos la lógica de negocio de la lógica de acceso a la base de datos.

Debe crearse una clase DAO que contenga todos los métodos de acceso a la base de datos. De este modo, si en el futuro cambiamos la base de datos, solo tendremos que modificar esta clase.

In [ ]:
pulic class DAO {
    private String url;
    private String usuario;
    private String password;

    public DAO(String url, String usuario, String password) {
        this.url = url;
        this.usuario = usuario;
        this.password = password;
    }

    public boolean insertarAlumno(String nombre, String apellido, LocalDate fechaNacimiento) throws SQLException {
        String sql = "INSERT INTO Alumnos (nombre, apellido, fecha_nacimiento) VALUES (?, ?, ?)";

        try (Connection conexion = DriverManager.getConnection(url, usuario, password); PreparedStatement sentencia = conexion.prepareStatement(sql);) {

            sentencia.setString(1, nombre);
            sentencia.setString(2, apellido);
            sentencia.setDate(3, Date.valueOf(fechaNacimiento));

            int cantidadFilas = sentencia.executeUpdate();
            System.out.printf("Insertadas %d filas\n", cantidadFilas);

            return cantidadFilas > 0;
        } catch (SQLException e) {
            System.out.println("Error al conectar a la base de datos: " + e.getMessage());
        }
    }

    public List<Alumno> obtenerAlumnosPorAnho(int anho) throws SQLException {
        String sql = "SELECT * FROM Alumnos WHERE YEAR(fecha_nacimiento) = ?";

        try (Connection conexion = DriverManager.getConnection(url, usuario, password); 
        PreparedStatement sentencia = conexion.prepareStatement(sql);) {
            sentencia.setInt(1, anho);

            try (ResultSet resultado = sentencia.executeQuery()) {
                List<Alumno> alumnos = new ArrayList<>();
                while (resultado.next()) {
                    Alumno alumno = new Alumno();
                    alumno.setNombre(resultado.getString("nombre"));
                    alumno.setApellido(resultado.getString("apellido"));
                    alumno.setFechaNacimiento(resultado.getDate("fecha_nacimiento").toLocalDate());
                    alumnos.add(alumno);
                }
                return alumnos;
            }
        } catch (SQLException e) {
            System.out.println("Error al conectar a la base de datos: " + e.getMessage());
        }
    }
}

De hecho, es aún más recomendable crear una interfaz DAO que contenga los métodos de acceso a la base de datos y una clase DAO que implemente dicha interfaz. En el programa principal, se creará un objeto de la interfaz DAO y se llamará a los métodos de la interfaz.

Así, si en el futuro cambiamos la base de datos, solo tendremos que modificar la clase DAO que implementa la interfaz.

In [ ]:
public interface DAO {
    public boolean insertarAlumno(String nombre, String apellido, LocalDate fechaNacimiento) throws SQLException;
    public List<Alumno> obtenerAlumnosPorAnho(int anho) throws SQLException;
}

public class DAOImplementacion implements DAO {
    String url;
    String usuario;
    String password;

    public DAOImplementacion(String url, String usuario, String password) {
        this.url = url;
        this.usuario = usuario;
        this.password = password;
    }

    @Override
    public boolean insertarAlumno(String nombre, String apellido, LocalDate fechaNacimiento) throws SQLException {
        String sql = "INSERT INTO Alumnos (nombre, apellido, fecha_nacimiento) VALUES (?, ?, ?)";

        try (Connection conexion = DriverManager.getConnection(url, usuario, password); PreparedStatement sentencia = conexion.prepareStatement(sql);) {

            sentencia.setString(1, nombre);
            sentencia.setString(2, apellido);
            sentencia.setDate(3, Date.valueOf(fechaNacimiento));

            int cantidadFilas = sentencia.executeUpdate();
            System.out.printf("Insertadas %d filas\n", cantidadFilas);

            return cantidadFilas > 0;
        }
    }

    @Override
    public List<Alumno> obtenerAlumnosPorAnho(int anho) throws SQLException {
        String sql = "SELECT * FROM Alumnos WHERE YEAR(fecha_nacimiento) = ?";

        try (Connection conexion = DriverManager.getConnection(url, usuario, password); 
        PreparedStatement sentencia = conexion.prepareStatement(sql);) {
            sentencia.setInt(1, anho);

            try (ResultSet resultado = sentencia.executeQuery()) {
                List<Alumno> alumnos = new ArrayList<>();
                while (resultado.next()) {
                    Alumno alumno = new Alumno();
                    alumno.setNombre(resultado.getString("nombre"));
                    alumno.setApellido(resultado.getString("apellido"));
                    alumno.setFechaNacimiento(resultado.getDate("fecha_nacimiento").toLocalDate());
                    alumnos.add(alumno);
                }
                return alumnos;
            }
        }
    }
}